In [0]:
!pip install -U PyYAML

Requirement already up-to-date: PyYAML in /usr/local/lib/python3.6/dist-packages (5.1.2)


In [0]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from keras.utils.np_utils import to_categorical
from keras.models import load_model
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
import tensorflow as tf



from tqdm import tqdm

import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
# from gensim.models import Word2Vec
# from gensim.models import KeyedVectors
import pickle

import os

from collections import Counter
from scipy.sparse import hstack

from prettytable import PrettyTable
import warnings

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = '/content/gdrive/My Drive/IRE_Major_Project/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", " will not ", phrase)
    phrase = re.sub(r"can\'t", " can not ", phrase)

    # general
    phrase = re.sub(r"n\'t", " not ", phrase)
    phrase = re.sub(r"\’re", " are ", phrase)
    phrase = re.sub(r"\’s", " is ", phrase)
    phrase = re.sub(r"\’d", " would ", phrase)
    phrase = re.sub(r"\’ll", " will ", phrase)
    phrase = re.sub(r"\’t", " not ", phrase)
    phrase = re.sub(r"\’ve", " have ", phrase)
    phrase = re.sub(r"\’m", " am ", phrase)
    return phrase

def remove_line_breaks(text):
  text = text.replace('\\r', ' ')
  text = text.replace('\\"', ' ')
  text = text.replace('\\n', ' ')
  return text

  # https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= { 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't","url","user"}

def perform_cleaning(text):
  phrase = text.lower().strip()
  # print(phrase)
  phrase = ' '.join(e for e in phrase.split() if e not in stopwords)
  phrase = re.sub('[^A-Za-z0-9]+', ' ', phrase)
  if not phrase:
    phrase = text.lower().strip()
  return phrase

def do_tokenize_lemmatize(text):
  lemmatizer = WordNetLemmatizer().lemmatize

  phrase = " ".join(lemmatizer(x,pos="v") for x in word_tokenize(text))
  if not phrase:
    phrase = text
  return phrase

def remove_user(text):
  return re.sub("@USER", '', text)


In [0]:
def get_user_mentions(text):
  return len(re.findall("@USER", text))

def remove_user(text):
  return re.sub("@USER", '', text)

def get_hash_tags(text):
  return len(re.findall("#", text))


def get_url_count(text):
  return len(re.findall("URL",text))




In [0]:
data = pd.read_csv(root_path+'preprocessed.csv')

In [0]:

data.head()

,id,tweet,new_tweet,user_mentions,n_hash_tags,n_urls,n_emojis,subtask_a,subtask_b,subtask_c,original_tweet_length,new_tweet_length
0,86426,@USER She should ask a few native Americans wh...,she ask native americans their take be,1,0,0,0,OFF,UNT,NaN,14,7
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,go home you drink maga trump2020,3,2,1,1,OFF,TIN,IND,11,6
2,16820,Amazon is investigating Chinese employees who ...,amazon investigate chinese employees sell inte...,0,5,1,0,NOT,NaN,NaN,27,19
3,62688,"@USER Someone should'veTaken"" this piece of sh...",someone should vetaken piece shit volcano,1,0,0,1,OFF,UNT,NaN,11,6
4,43605,@USER @USER Obama wanted liberals &amp; illega...,obama want liberals amp illegals move red state,2,0,0,0,NOT,NaN,NaN,12,8


Dropping subtask a and c columns

# Splitting Data: Train and Test


In [0]:
Y = data[['subtask_b'  , 'subtask_a' ,  'subtask_c']]
X = data.drop(['subtask_b' , 'subtask_a' , "subtask_c" ,'id'],axis=1)
print("Shape of X: ",X.shape)
print("Shape of Y: ",Y.shape)

Shape of X:  (13240, 8)
Shape of Y:  (13240, 3)


#### Change the mapping of the label column to binary


In [0]:
Y['subtask_a'] = Y['subtask_a'].map({'OFF' : 'OFFENIVE' , 'NOT' : 'NOT OFFENSIVE'})
Y['subtask_b'] = Y['subtask_b'].map({'UNT' : 'NOT TARGATED' , 'TIN' : 'TARGATED'})
Y['subtask_c'] = Y['subtask_c'].map({'OTH' : 'OTHER' , 'IND' : 'INDIVIDUAL' , 'GRP' : "GROUP"})


In [0]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     print(X_train.isnull())
#  X_train[X_train.isna().any(axis=1)]

# Convolution Model

In [0]:
from keras.models import Model
from keras.optimizers import SGD, Adam
from keras.layers import Input, Dense, Dropout, Flatten, Lambda, Embedding
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from keras.initializers import RandomNormal
from keras.engine import Layer, InputSpec
from keras import backend as K

## Evaluation Metrics

## Making Data Model Ready: Encoding Tweets

### Quantization  

In [0]:
def encode_data(x, maxlen, vocab):
    # Iterate over the loaded data and create a matrix of size (len(x), maxlen)
    # Each character is encoded into a one-hot array later at the lambda layer.
    # Chars not in the vocab are encoded as -1, into an all zero vector.
    
    input_data = np.zeros((len(x), maxlen), dtype=np.int)
    for dix, sent in enumerate(x):
        counter = 0
        for c in sent:
            if counter >= maxlen:
                pass
            else:
                ix = vocab.get(c, -1)  # get index from vocab dictionary, if not in vocab, return -1
                input_data[dix, counter] = ix
                counter += 1
    return input_data


def create_vocab_set():
    # This alphabet is 69 chars vs. 70 reported in the paper since they include two
    # '-' characters. See https://github.com/zhangxiangxiao/Crepe#issues.

    alphabet = set(list(string.ascii_lowercase) + list(string.digits) +
                   list(string.punctuation) + ['\n'])
    vocab_size = len(alphabet)
    vocab = {}
    reverse_vocab = {}
    for ix, t in enumerate(alphabet):
        vocab[t] = ix
        reverse_vocab[ix] = t

    return vocab, reverse_vocab, vocab_size, alphabet

In [0]:
vocab, reverse_vocab, vocab_size, alphabet = create_vocab_set()


## feature set

In [0]:
f11 = encode_data(X['new_tweet'], 1500 , vocab)
f12 = encode_data(X['new_tweet'], 1014 , vocab)
f13 = encode_data(X['new_tweet'], 1200 , vocab)

f2 = X['user_mentions'].values.reshape(-1,1)
f3 = X['n_hash_tags'].values.reshape(-1,1)
f4 = X['n_urls'].values.reshape(-1,1)
f5 = X['n_emojis'].values.reshape(-1,1)
f6 = X['new_tweet_length'].values.reshape(-1,1)
f7 = X['original_tweet_length'].values.reshape(-1,1)

X_encoded1 = np.concatenate((f11 , f2 , f3 , f4 , f7 ) , axis = 1)
X_encoded2 = np.concatenate((f12 , f2 , f3 , f4 , f5 , f6 , f7 ) , axis = 1)
X_encoded3 = np.concatenate((f13 , f2 , f3 , f4 , f6 ) , axis = 1)

print(X_encoded1.shape)
print(X_encoded2.shape)
print(X_encoded3.shape)

(13240, 1504)
(13240, 1020)
(13240, 1204)


In [0]:
print(f11)


[[62 35 27 ...  0  0  0]
 [13 66 -1 ...  0  0  0]
 [ 0 14  0 ...  0  0  0]
 ...
 [56 27 43 ...  0  0  0]
 [43 11 62 ...  0  0  0]
 [62 43  0 ...  0  0  0]]


In [0]:
print(X_encoded1)

[[62 35 27 ...  0  0 14]
 [13 66 -1 ...  2  1 11]
 [ 0 14  0 ...  5  1 27]
 ...
 [56 27 43 ...  0  0 11]
 [43 11 62 ...  0  0  2]
 [62 43  0 ...  9  1 30]]


In [0]:
## Loading all saved model

from keras.models import model_from_yaml
# import yaml

# from keras.models import load_model


yaml_file1 = open(root_path + '/CNN/model_A.yaml', 'r')
loaded_model_yaml1 = yaml_file1.read()
# yaml_file1.close()
model1 = model_from_yaml(loaded_model_yaml1)
# load weights into new model
model1.load_weights(root_path + "/CNN/model_A.h5")


yaml_file2 = open(root_path + '/CNN/model_B.yaml', 'r')
loaded_model_yaml2 = yaml_file2.read()
yaml_file2.close()

model2 = model_from_yaml(loaded_model_yaml2)
# load weights into new model
model2.load_weights(root_path + "/CNN/model_B.h5")


yaml_file3 = open(root_path + '/CNN/model_C.yaml', 'r')
loaded_model_yaml3 = yaml_file3.read()
yaml_file3.close()
model3 = model_from_yaml(loaded_model_yaml3)
# load weights into new model
model3.load_weights(root_path + "/CNN/model_C.h5")
print("Loaded model from disk")


Loaded model from disk


# Predictions

In [0]:
def predict(sample1, sample2 , sample3) :
  res1 = model1.predict(sample1)
  res1_cat = res1.argmax(axis = 1)

  if res1_cat[0] == 0 :
    print("Subtask A Result :  NOT OFFENSIVE")
    return

  res2 = model2.predict(sample2)
  res2_cat = res2.argmax(axis = 1)

  if res2_cat[0] == 0 :
    print("Subtask A Result : OFFENSIVE")
    print("Subtask B Result : NOT TARGETED")
    return 

  res3 = model3.predict(sample3)
  res3_cat = res3.argmax(axis = 1)

  print("Subtask A Result : OFFENSIVE")
  print("Subtask B Result : TARGETED")
  if res3_cat[0] == 0 : 
    print("Subtask C Result : OTHER")
  elif res3_cat[0] == 1 :
    print("Subtask C Result : INDIVIDUAL")
  else :
    print("Subtask C Result : GROUP")


In [0]:
import re
def unseen_predict(tweet) :
  new_tweet = decontracted(tweet.lower())
  new_tweet = remove_line_breaks(new_tweet)
  new_tweet = remove_user(new_tweet)
  new_tweet = perform_cleaning(new_tweet)
  new_tweet = do_tokenize_lemmatize(new_tweet)

  n_tweet = [new_tweet]
  f2 = [[get_user_mentions(tweet)]]
  f3 = [[get_hash_tags(tweet)]]
  f4 = [[get_url_count(tweet)]]
  f5 =[[0]]
  f6 = [[len(new_tweet.split())]]
  f7 = [[len(tweet.lower().split())]]
  # print(new_tweet)
  x = encode_data(n_tweet, 1500 , vocab)
  encoding1 = np.concatenate((x , f2 , f3 , f4 , f7) , axis = 1) 
  # print(x)
  encoding2 = np.concatenate((encode_data(n_tweet, 1014 , vocab) , f2 , f3 , f4 , f5 ,f6 , f7) , axis = 1) 
  encoding3 = np.concatenate((encode_data(n_tweet, 1200 , vocab) , f2 , f3 , f4 , f6) , axis = 1) 


  predict(encoding1 , encoding2 , encoding3)


## FINAL PREDICTIONS

In [276]:
for ind in range(1600,1602):
  print("tweet is :  ",  X["tweet"][ind] )

  print("\n\npredictions are :  \n")
  predict(X_encoded1[ind:ind+1] , X_encoded2[ind:ind+1] , X_encoded3[ind:ind+1] )

  print("\n\nOriginal Labels are  : \n")
  # print(Y.iloc[ind , :])


tweet is :   @USER She don’t even wear the shit I buy her now. Move away from me.


predictions are :  

Subtask A Result :  NOT OFFENSIVE


Original Labels are  : 

tweet is :   @USER @USER Chequers delivers Brexit (changes the relationship with the EU). It does not deliver leave the European Union" (referendum question.) I think we should stop using the word Brexit to mean leave, and actually say "leave the EU". "Brexit" meaning willfully mutated by remainers."


predictions are :  

Subtask A Result :  NOT OFFENSIVE


Original Labels are  : 



# LIVE DEMO

In [283]:
# tweet = "You're another hypocrite"
# tweet = "he is useless"
# tweet = "@USER I suppose you think the way liberals attacked Sarah Palin was courageous also. You’re just another hypocrite."
# tweet = "@USER Bravo. Down with CNN."
# tweet = "@USER Never nigga 💀💀"
# tweet = "@USER @USER Love you Norm!"
unseen_predict(tweet)

Subtask A Result :  NOT OFFENSIVE
